1. !pip install selenium
2. https://chromedriver.chromium.org/downloads
3. chrome://version/ 버전확인

- 웹사이트 요청 -> 요청된 페이지가 전체 다운로드 -> 문서화 -> 원하는 태그 값을 찾는다
- selenium : 웹사이트 요청 -> 크롬브라우저를 가상메모리 통해 연동 -> 원하는 태그 값을 찾는다

In [ ]:
# !pip install selenium

In [8]:
from selenium import webdriver  # selenium library에서 webdriver 함수를 import한다
from selenium.webdriver.common.keys import Keys

# Chrome를 조작
# class selenium.webdriver.chrome.webdriver.WebDrive
driver = webdriver.Chrome(executable_path = "C:\\Users\\USER\\desktop\\programming\\chromedriver")
driver.get('http://www.python.org')

# assert "python" not in driver.title
# print(driver.title)

In [9]:
from selenium import webdriver
from bs4 import BeautifulSoup

driver = webdriver.Chrome(executable_path = "C:\\Users\\USER\\desktop\\programming\\chromedriver")
driver.implicitly_wait(3)

driver.get('https://logins.daum.net/accounts/signinform.do?url=https%3A%2F%2Fwww.daum.net%2F')

driver.find_element_by_name('id').send_keys('본인의 아이디')
driver.find_element_by_name('pw').send_keys('본인의 비밀번호')
driver.find_element_by_id('loginBtn').click()

In [ ]:
import time
import math
import re
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

print("검색할 영화")
query = input()
driver = webdriver.Chrome(executable_path = "C:\\Users\\USER\\desktop\\programming\\chromedriver")

driver.get('https://movie.naver.com/')

# 1) 검색 리뷰수를 입력해서 구현해보자
cnt = int(input("검색 리뷰수 : "))
page_cnt = math.ceil(cnt / 10)

# driver.implicitly_wait(10)
time.sleep(10)

# 검색어 입력하는 id를 찾아오자
element = driver.find_element_by_id("ipt_tx_srch")
element.click()
element.send_keys(query)

driver.find_element_by_xpath('//*[@id="jSearchArea"]/div/button').click()
driver.find_element_by_xpath('//*[@id="old_content"]/ul[2]/li[1]/p').click()
driver.find_element_by_xpath('//*[@id="movieEndTabMenu"]/li[5]/a').click()

driver.switch_to.frame('pointAfterListIframe')  # 변동되는 동적 프레임을 호출 하게 되면 내용이 바뀌어도 바뀐 데이터를 리턴

# 2) 동기화 및 요청 건수 비교
html = driver.page_source  # 문서화한다
soup = BeautifulSoup(html, "html.parser")

result = soup.find('div', class_ = 'score_total').find('strong', class_ = 'total').find_all('em')
result2 = result[0].get_text()

# 3) 전체 검색 결과 수 / 실제 최종 출력 수 (관람객 평점)
result3 = result2.replace(",","")
result4 = re.search("\d+", result3)  # 검증
search_cnt = int(result4.group())

if cnt > search_cnt:
    cnt = search_cnt

print("전체 검색 결과 건수 : " + str(search_cnt))
print("실제 최종 출력 수 : " + str(cnt))

# 4) 별점 / 리뷰 내용 / 작성자 / 작성일자 / 공감 횟수 / 비공감 횟수
score2 = []  # 별점(평점)
review2 = []  # 리뷰 내용
writer2 = []  # 작성자
wdate2 = []  # 작성일자
g_gogam2 = []  # 공감 횟수
b_gogam2 = []  # 비공감 횟수
count = 0

for x in range(1, page_cnt + 1):
    f = open("naver_api.txt", 'a', encoding = 'utf-8')
    
    html = driver.page_source  # 문서화한다
    soup = BeautifulSoup(html, "html.parser")
    
    score_result = soup.find('div', class_ = 'score_result').find('ul')
    slist = score_result.find_all('li')
    
    for li in slist:
        count += 1
        print("%s 번째 내용" % count + '\n')
        
        f.write('\n')
        f.write("%s 번째 내용" % count + '\n')
        
        # 별점
        score = li.find('div', class_ = 'star_score').get_text().replace('\n','')
        print("1. 별점(평점) : " + score + '\n')
        f.write("1. 별점(평점) : " + score + '\n')
        score2.append(score)
        
        # 리뷰 내용
        review = li.find('div', class_ = 'score_reple').find('p').get_text()
        print("2. 리뷰 내용 : " + review + '\n')
        f.write("2. 리뷰 내용 : " + review + '\n')
        review2.append(review)
        
        # 작성자
        writer = li.find('div', class_ = 'score_reple').find_all('em')
        writer02 = writer[0].find('span').get_text()
        print("3. 작성자 : " + writer02 + '\n')
        f.write("3. 작성자 : " + writer02 + '\n')
        writer2.append(writer02)
        
        # 작성일자
        wdate = li.find('div', class_ = 'score_reple').find_all('em')[1].get_text()
        print("4. 작성일자 : " + wdate + '\n')
        f.write("4. 작성일자 : " + wdate + '\n')
        wdate2.append(wdate)
        
        # 공감 횟수
        g_gogam = li.find('div', class_ = 'btn_area').find_all('strong')[0].get_text()
        print("5. 공감 횟수 : " + g_gogam + '\n')
        f.write("5. 공감 횟수 : " + g_gogam + '\n')
        g_gogam2.append(g_gogam)
        
        # 비공감 횟수
        b_gogam = li.find('div', class_ = 'btn_area').find_all('strong')[1].get_text()
        print("6. 비공감 횟수 : " + b_gogam + '\n')
        f.write("6. 비공감 횟수 : " + b_gogam + '\n')
        b_gogam2.append(b_gogam)
        
        if count >= cnt:
            break
            
    f.close()        
    time.sleep(2)
    x += 1
    if x > page_cnt:
        break
        
    driver.find_element_by_xpath('//*[@id="pagerTagAnchor%s"]' % x).click()  # 페이지 이동값
    
# 5) 판다스로 연동
movie = pd.DataFrame()
movie['별점(평점)'] = pd.Series(score2)
movie['리뷰 내용'] = pd.Series(review2)
movie['작성자'] = pd.Series(writer2)
movie['작성일자'] = pd.Series(wdate2)
movie['공감 횟수'] = pd.Series(g_gogam2)
movie['비공감 횟수'] = pd.Series(b_gogam2)

# 파일로 저장
movie.to_csv('naver_movie.csv', encoding='utf-8', index=True)

# 엑셀로 저장
# movie.to_excel('naver_movie.xls', index=True)

# 8) 드라이버 종료
driver.close()